This notebook creates a dataset of iDigBio records using the following algorithm:

```
parameter: target_total_family_count
parameter: target_family_record_count

sorted_phyla <- sort phyla by *ascending* family count

num_phyla_remaining <- size(phyla)
for each phylum in sorted_phyla {
    target_family_count <- floor(target_total_family_count / num_phyla_remaining)
    family_record_counts <- count records for each family
    families <- get families under phylum with sufficient number of records

    family_bins <- place families into bins of similar record counts
    sorted_family_bins <- sort bins by *ascending* family count

    num_bins_remaining <- size(family_bins)
    for each bin in sorted_family_bins {
        target_family_count <- floor(target_family_count / num_bins_remaining)
        sorted_families <- sort families in bin by *descending* record count
        
        family_count <- 0
        for family in bin while family_count < target_family_count {
            family_records <- get diverse sample of records for family
            if size(family_records) > target_family_record_count {
                save family_records
                increment family_count
            }
        }

        decrement num_bins_remaining
    }

    decrement num_phyla_remaining
}
```

Algorithm for sampling a diverse set of records for each family:
* Get list of unique species, county pairs
* Collect one record per pair

In [1]:
if "snakemake" in globals():
    print("Parameters:", dict(snakemake.params))
    records_path = snakemake.output[0]
    kingdom = snakemake.params.kingdom
    phyla = snakemake.params.phyla
    min_records_per_family = snakemake.params.min_records_per_family
    max_records_per_family = snakemake.params.max_records_per_family
    families_per_kingdom = snakemake.params.families_per_kingdom
    required_fields = snakemake.params.required_fields
else:
    records_path = "test.jsonl"
    kingdom = "plantae"
    phyla = {"tracheophyta", "bryophyta", "marchantiophyta", "rhodophyta", "chlorophyta", "charophyta", "anthocerotophyta"}
    min_records_per_family = 25
    max_records_per_family = 200
    families_per_kingdom = 168
    required_fields = ["kingdom", "family", "genus", "specificepithet", "countrycode", "stateprovince", "county"]
    

In [2]:
print(f"Records per kingdom: between {min_records_per_family * families_per_kingdom} and {max_records_per_family * families_per_kingdom}")
print("Families per kingdom:", families_per_kingdom)

phyla_in_kingdom = len(phyla)
families_per_phylum = int(families_per_kingdom / phyla_in_kingdom)
print("Number of phyla:", phyla_in_kingdom)
print("Families per phylum:", families_per_phylum)

Records per kingdom: between 4200 and 33600
Families per kingdom: 168
Number of phyla: 7
Families per phylum: 24


In [3]:
import json
import requests
import math
import numpy as np
from attr import dataclass

api = "https://beta-search.idigbio.org/v2"

required_fields_rq = {field: {"type": "exists"} for field in required_fields}
default_rq = required_fields_rq | {
    "taxonrank": "species"
}

def get_idigbio_summary(rq, top_fields, count, rf=required_fields, verbose=False):
    response = requests.post(f"{api}/summary/top/records", json=dict(
        rq=default_rq | rq,
        top_fields=top_fields,
        count=count
    ))

    return response.json()[top_fields[0]]

# limit=5000 is the max allowed by the APIs
def get_idigbio_records(rq, limit=5000, gt_uuid=None, api=api):
    data = {
        "rq": default_rq | rq,
        "limit": limit,
    }

    if gt_uuid is not None:
        data["rq"] |= {"uuid": {"type": "range", "gt": gt_uuid}}
        data |= {
            "sort": [{ "uuid": "asc" }]
        }

    response = requests.post(f"{api}/search/records", json=data)

    return response.json()

In [4]:
import re

class SpeciesCounty():
    species: str
    county: str

    bad_names = {
        "incertae sedis",
        "na",
        "no",
        "no data",
        "no disponible",
        "not available",
        "not given",
        "not specified",
        "not stated",
        "undetermined",
        "unknown",
        "unspecified",
        "unstated",
        "species",
        "scientific name",
        "country",
        "state",
        "province",
        "stateprovince",
        "county",
    }

    def _get_simplified_species_name_parts(self):
        return [s for s in self.species.split() if (
            len(s) > 0 and
            s[0].isalnum()
        )][:2]

    def _get_simplified_county_name_parts(self):
        county = (
            re.sub("[\\(\\[].*?[\\)\\]]", "", self.county)
            .replace(".", "")
        )

        raw_parts = [s for s in county.split()]
        return [s for s in raw_parts if (
            len(s) > 1 and
            s[0].isalnum() and
            s not in {
                "co",
                "county",
                "pref",
                "prefecture",
                "i",
                "island",
                "mun",
                "municipio",
                "municipality",
                "dis",
                "dist",
                "distr"
                "district",
                "div",
                "division"
                "reg",
                "region",
                "group",
                "prov",
                "province",
                "rayon", # Russia
                "raion",
                "kn", # Sweden
                "department",
                "par",
                "parish",
                "bezirk", # Switzerland
                "census",
                "area",
                "land",
                "council",
                "admin"
                "admin.",
                "administrative",
                "administration",
                "administrasi",
                "metro",
                "metro.",
                "metropolitan",
                "barrio",
                "burrow",
                "borough",
                "raya",
                "regency",
                "kabupaten",
                "auto",
                "autonomous"
            }
        )]
    
    def __init__(self, species, county):
        self.species = species.lower()
        self.county = county.lower()

        species_name_parts = self._get_simplified_species_name_parts()
        county_name_parts = self._get_simplified_county_name_parts()

        self.simple_species_name = " ".join(species_name_parts)
        self.simple_county_name = " ".join(county_name_parts)

        self.valid = (
            self.simple_species_name not in self.bad_names and
            self.simple_county_name not in self.bad_names and
            len(species_name_parts) == 2 and
            len(county_name_parts) > 0
        )

    def ok(self):
        return self.valid

    def __hash__(self):
        """
        Use clean county names to test equivalence, but preserve the original uncleaned names to
        maintain record retrievability
        """
        return hash(f"{self.simple_species_name}\t{self.simple_county_name}")

    def __eq__(self, value: object) -> bool:
        return isinstance(value, self.__class__) and hash(self) == hash(value)
    
    def __repr__(self) -> str:
        return f"SpeciesCounty({self.county}, {self.species})"


assert len(set([
    SpeciesCounty("big bug", "alachua"),
    SpeciesCounty("big bug l.", "alachua co."),
    SpeciesCounty("big bug l. 1788", "alachua county [but not really]"),
])) == 1

assert SpeciesCounty("[the species]", "hah").ok() == False
assert SpeciesCounty("the species", "[hah]").ok() == False
assert SpeciesCounty("incertae sedis", "baker").ok() == False

In [5]:
def get_family_species_county_pairs(kingdom, family):
    family_summary = get_idigbio_summary(
        rq= {
            "kingdom": kingdom,
            "family": family,
            "basisofrecord": "preservedspecimen",
            "taxonrank": "species"
        },
        top_fields=["phylum", "scientificname", "county"],
        count=max_records_per_family,
        verbose=False
    )

    unique_species_county_pairs = {
        sc
        for family, family_data in family_summary.items()
        for species, species_data in family_data["scientificname"].items()
        for county, county_data in species_data["county"].items()
        for sc in (SpeciesCounty(species.lower(), county.lower()),)
        if sc.ok()
    }

    return list(unique_species_county_pairs)


list(get_family_species_county_pairs("plantae", "stereodontaceae"))[2]

SpeciesCounty(svennevad, hypnum pratense)

In [6]:
def get_record_for_species_county(kingdom: str, species_county: SpeciesCounty) -> dict:
    return get_idigbio_records(
        rq={
            "kingdom": kingdom,
            "scientificname": species_county.species,
            "county": species_county.county
        },
        limit=1
    )["items"][0]


def get_species_county_records(kingdom, sc_pairs, count):
    shuffle_index = np.random.permutation(len(sc_pairs))[:count]
    for sc in (sc_pairs[i] for i in shuffle_index):
        yield get_record_for_species_county(kingdom, sc)


# Example use:
if True:
    sc_pairs = get_family_species_county_pairs("plantae", "stereodontaceae")
    print(next(iter(get_species_county_records("plantae", sc_pairs, 1))).keys())

dict_keys(['uuid', 'type', 'etag', 'data', 'indexTerms'])


In [7]:
phylum_summaries = get_idigbio_summary(
    rq={
        "kingdom": kingdom,
        "phylum": list(phyla)
    },
    top_fields=["phylum", "family"],
    count=families_per_kingdom,
    verbose=False
)

# For families that appear in more than one phylum, assign them to the phylum for which
# they have the most records
all_family_phylum_assignments = dict()
family_record_counts_by_phylum = dict()

for phylum, data in phylum_summaries.items():
    family_counts = dict(map(lambda x: (x[0], x[1]["itemCount"]), data["family"].items()))

    for family, count in family_counts.items():
        current_count = family_record_counts_by_phylum.get(family, 0)
        if count > current_count:
            all_family_phylum_assignments[family] = phylum
            family_record_counts_by_phylum[family] = count

In [8]:
def get_family_candidates(phylum, phylum_family_counts):
    bad_family_names = {
        "",
        "\"\"",
        "unknown",
        "unplaced county",
    }
    
    def check_family_name(family_name: str, family_summary: dict):
        return (family_name.lower() not in bad_family_names and
                all_family_phylum_assignments[family_name] == phylum and
                family_summary["itemCount"] >= min_records_per_family)
    
    return {f: v["itemCount"] for f, v in phylum_family_counts.items() if check_family_name(f, v)}


# 1. For each phylum
# - Get family candidates
# - Count family candidates
phylum_family_candidates = {phylum: get_family_candidates(phylum, data["family"]) for phylum, data in phylum_summaries.items()}
phylum_family_counts = {phylum: len(family_counts) for phylum, family_counts in phylum_family_candidates.items()}

# 2. Sort phyla by family count, ascending
phylum_family_counts = dict(sorted(phylum_family_counts.items(), key=lambda kv: (kv[1], kv[0])))
phylum_family_counts

{'anthocerotophyta': 4,
 'charophyta': 9,
 'chlorophyta': 60,
 'marchantiophyta': 65,
 'rhodophyta': 79,
 'bryophyta': 122,
 'tracheophyta': 168}

In [9]:
def gen_family_bins_with_ascending_size(family_sc_counts, num_bins):
    sc_counts = list(family_sc_counts.values())

    if len(sc_counts) == 0:
        return []

    # Bin families by record count
    bin_min: int = min(sc_counts) - 1
    bin_max: int = max(sc_counts) + 1
    bin_edges = np.linspace(bin_min, bin_max, num_bins + 1)

    # Get the bin each family falls into
    family_bin_assignments: np.ndarray = np.digitize(sc_counts, bin_edges) - 1
    
    # Return families in each bucket
    for bin_index in range(num_bins):
        family_names_in_bin = np.array(list(family_sc_counts.keys()))\
            [family_bin_assignments == bin_index]
        
        if len(family_names_in_bin) > 0:
            family_counts_in_bin = np.array(list(family_sc_counts.values()))\
                [family_bin_assignments == bin_index]
        
            yield dict(zip(
                map(str, family_names_in_bin),
                map(int, family_counts_in_bin)
            ))

In [10]:
def iter_families_for_phylum(sc_counts_by_family, target_family_count):
    num_bins = target_family_count
    num_families_remaining_for_phylum = target_family_count

    for j, family_bin_sc_counts in enumerate(gen_family_bins_with_ascending_size(sc_counts_by_family, num_bins)):
        num_bins_remaining = num_bins - j
        max_num_families_per_bucket = math.floor(num_families_remaining_for_phylum / num_bins_remaining)

        # Sort families descending by [(species, county)] count
        family_bin_sc_counts = list(sorted(family_bin_sc_counts.items(), key=lambda kv: (kv[1], kv[0]), reverse=True))\
            [:max_num_families_per_bucket]
        
        bin_families, _ = zip(*family_bin_sc_counts)
        yield from bin_families
        
        num_families_remaining_for_phylum -= len(bin_families)

In [11]:
num_families_remaining = families_per_kingdom

with open(records_path, "w") as f:
    for i, (phylum, phylum_family_count) in enumerate(phylum_family_counts.items()):
        num_phyla_remaining = len(phyla) - i
        target_family_count = math.floor(num_families_remaining / num_phyla_remaining)

        print(f"Phylum {i+1:<2}: {phylum} (max {target_family_count} familiies)")

        species_counties_by_family = {
            family: get_family_species_county_pairs(kingdom, family)
            for family in phylum_family_candidates[phylum]
        }

        # Filter out families with too few records
        sc_counts_by_family = {
            family: len(scs)
            for family, scs in species_counties_by_family.items()
            if len(scs) >= min_records_per_family
        }

        phylum_family_count = 0
        for family in iter_families_for_phylum(sc_counts_by_family, target_family_count):
            print(f"- {family}")
            phylum_family_count += 1

            for record in get_species_county_records(
                    kingdom, 
                    species_counties_by_family[family],
                    max_records_per_family
                ):
                json.dump(record, f)
                f.write("\n")
        
        num_families_remaining -= phylum_family_count
        print(f"  {phylum_family_count} families sampled")

Phylum 1 : anthocerotophyta (max 24 familiies)
- dendrocerotaceae
- anthocerotaceae
- notothyladaceae
  3 families sampled
Phylum 2 : charophyta (max 27 familiies)
- mesotaeniaceae
- zygnemataceae


KeyboardInterrupt: 